In [10]:
import pandas as pd
import numpy as np
from sklearn.ensemble import AdaBoostClassifier, StackingClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import accuracy_score, confusion_matrix

# Part 3

df = pd.read_csv('train_data_with_groups.csv')
x_train, x_test, y_train, y_test = train_test_split(df.drop(['Bankrupt?', 'Group'], axis=1), df['Group'], train_size=0.8, random_state=42)

df = df[df['Group'] == 1]
x_train, x_test, y_train, y_test = train_test_split(df.drop(['Bankrupt?', 'Group'], axis=1), df['Bankrupt?'], train_size=0.8, random_state=42)

# New models
model1 = AdaBoostClassifier(n_estimators=100, random_state=42).fit(x_train, y_train)
model2 = GradientBoostingClassifier(n_estimators=100, random_state=42).fit(x_train, y_train)
model3 = KNeighborsClassifier().fit(x_train, y_train)

model1_pred = model1.predict(x_test)
model2_pred = model2.predict(x_test)
model3_pred = model3.predict(x_test)

print('Model 1 (AdaBoost) accuracy:', accuracy_score(y_test, model1_pred))
print('Model 2 (Gradient Boosting) accuracy:', accuracy_score(y_test, model2_pred))
print('Model 3 (KNeighbors) accuracy:', accuracy_score(y_test, model3_pred))
print((accuracy_score(y_test, model1_pred)+accuracy_score(y_test, model2_pred)+accuracy_score(y_test, model3_pred))/3)
print(confusion_matrix(y_test, model1_pred))
print(confusion_matrix(y_test, model2_pred))
print(confusion_matrix(y_test, model3_pred))
# Printing the 5 most important features for AdaBoostClassifier
print('Top 5 Important Features for AdaBoostClassifier:')
feature_importance_model1 = list(zip(x_train.columns, model1.feature_importances_))
feature_importance_model1.sort(key=lambda x: x[1], reverse=True)
for i, (feature, importance) in enumerate(feature_importance_model1[:5]):
    print(f'{i+1}. Feature: {feature}, Importance: {importance:.4f}')

# Printing the 5 most important features for GradientBoostingClassifier
print('Top 5 Important Features for GradientBoostingClassifier:')
feature_importance_model2 = list(zip(x_train.columns, model2.feature_importances_))
feature_importance_model2.sort(key=lambda x: x[1], reverse=True)
for i, (feature, importance) in enumerate(feature_importance_model2[:5]):
    print(f'{i+1}. Feature: {feature}, Importance: {importance:.4f}')

combined_pred = np.column_stack((model1_pred, model2_pred, model3_pred))
metaModel = LogisticRegression().fit(combined_pred, y_test)
y_pred = metaModel.predict(combined_pred)
print('Meta Model Accuracy:', accuracy_score(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))



C:\Users\edwar\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\ensemble\_weight_boosting.py:519: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(


Model 1 (AdaBoost) accuracy: 0.925531914893617
Model 2 (Gradient Boosting) accuracy: 0.8617021276595744
Model 3 (KNeighbors) accuracy: 0.9361702127659575
0.9078014184397163
[[85  3]
 [ 4  2]]
[[80  8]
 [ 5  1]]
[[88  0]
 [ 6  0]]
Top 5 Important Features for AdaBoostClassifier:
1. Feature:  Cash/Total Assets, Importance: 0.0900
2. Feature:  Operating profit per person, Importance: 0.0800
3. Feature:  Realized Sales Gross Margin, Importance: 0.0500
4. Feature:  Persistent EPS in the Last Four Seasons, Importance: 0.0500
5. Feature:  Cash Flow Per Share, Importance: 0.0500
Top 5 Important Features for GradientBoostingClassifier:
1. Feature:  Working Capital/Equity, Importance: 0.2359
2. Feature:  Cash Flow to Total Assets, Importance: 0.0880
3. Feature:  Operating profit per person, Importance: 0.0876
4. Feature:  Cash/Current Liability, Importance: 0.0802
5. Feature:  Quick Assets/Total Assets, Importance: 0.0421
Meta Model Accuracy: 0.9361702127659575
[[88  0]
 [ 6  0]]


In [11]:
# Part 4

df = pd.read_csv('train_data_with_groups.csv')

# New stacking classifier with different estimators
stacking_clf = StackingClassifier(
    estimators=[
        ('lr', LogisticRegression(random_state=42)),
        ('ab', AdaBoostClassifier(random_state=42)),
        ('knn', KNeighborsClassifier())
    ],
    final_estimator=GradientBoostingClassifier(random_state=43),
    cv=10
)

stacking_clf.fit(x_train, y_train)

y_pred = stacking_clf.predict(x_test)
print('Stacking Classifier Accuracy:', accuracy_score(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))

# Part 4 - Cross-validation with new stacking classifier
cross_val_scores = cross_val_score(stacking_clf, x_train, y_train, cv=10, scoring='accuracy')
print("Cross-Validation Scores:", np.mean(cross_val_scores))
print(df.shape[1])


C:\Users\edwar\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\ensemble\_weight_boosting.py:519: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(
C:\Users\edwar\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\ensemble\_weight_boosting.py:519: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(
C:\Users\edwar\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\ensemble\_weight_boosting.py:519: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent th

Stacking Classifier Accuracy: 0.9361702127659575
[[85  3]
 [ 3  3]]


C:\Users\edwar\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\ensemble\_weight_boosting.py:519: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(
C:\Users\edwar\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\ensemble\_weight_boosting.py:519: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(
C:\Users\edwar\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\ensemble\_weight_boosting.py:519: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent th

Cross-Validation Scores: 0.9172119487908963
47
